# Aries Basic Controller - Issuer Example

This notebook walks through how to issue a credential across a previously established connection.

**Note: You should first complete the DIDExchange example notebooks to establish a connection between Alice and Bob**

* [Alice](http://localhost:8888/notebooks/did-exchange-inviter.ipynb)
* [Bob](http://localhost:8889/notebooks/did-exchange-invitee.ipynb)


In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)
    

## Register Listeners

Make sure you have terminated the agent controller being used in the did-exchange example or it will throw an error.

In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())



<Task pending coro=<AriesAgentController.listen_webhooks() running at /workspace/aries_basic_controller/aries_controller.py:64>>

In [4]:
def cred_handler(payload):
    print("Handle Credentials", payload)
    
cred_listener = {
    "topic": "issue-credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

## Check the agent has an active connection

If no connections returned, or the connection not active then this tutorial will not work. You will need to revisit the did-exchange notebooks for Alice and Bob.

In [5]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")
    


Results :  [{'updated_at': '2020-07-08 15:34:56.973173Z', 'created_at': '2020-07-08 15:34:46.693070Z', 'initiator': 'external', 'invitation_key': 'G3Nvbw1tUTHxhYRuLfRMviFqpeQ18DXovgK2L8TgivmB', 'routing_state': 'none', 'state': 'active', 'my_did': '98dEW7uzMQTJN3U5hTffhw', 'their_did': 'BWxbPo7p7TguRhu6QBVdEn', 'connection_id': 'ad11f6e5-351d-459f-aedc-f571ab374fa4', 'accept': 'manual', 'their_label': 'Bob', 'request_id': '70b86320-95b0-44f5-9677-e3ffc91ff81f', 'invitation_mode': 'once'}]
Connection : {'updated_at': '2020-07-08 15:34:56.973173Z', 'created_at': '2020-07-08 15:34:46.693070Z', 'initiator': 'external', 'invitation_key': 'G3Nvbw1tUTHxhYRuLfRMviFqpeQ18DXovgK2L8TgivmB', 'routing_state': 'none', 'state': 'active', 'my_did': '98dEW7uzMQTJN3U5hTffhw', 'their_did': 'BWxbPo7p7TguRhu6QBVdEn', 'connection_id': 'ad11f6e5-351d-459f-aedc-f571ab374fa4', 'accept': 'manual', 'their_label': 'Bob', 'request_id': '70b86320-95b0-44f5-9677-e3ffc91ff81f', 'invitation_mode': 'once'}
Active Conne

## Write a Schema to the Ledger

For more details see the [schema-api notebook](http://localhost:8888/notebooks/schema_api.ipynb)

In [6]:
# Define you schema name - must be unique on the ledger
schema_name = "open_mined_contributor"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["name", "skill"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)


PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1


## Write a Credential Definition to the Ledger

More details in the [definitions notebook](http://localhost:8888/notebooks/definitions_api.ipynb)

In [7]:
response = await agent_controller.definitions.write_cred_def(schema_id)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)

PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default


**Note: You should be able to see both schema and definitions transactions on the local network [here](http://localhost:9000)**

## Populate the Attribues to Issue to Bob

The schema defines two attributes: name and skill. Both must be populated with attribute values that Alice wishes to issue in a credential to Bob. To do this a list of objects must be created, each object containing the name of the attribute and it's value at a minimum. You can set the values to anything you wish.

TODO: Some additional fields such as mime-type can be defined.

In [8]:
credential_attributes = [
    {"name": "name", "value": "Bob"},
    {"name": "skill", "value": "researcher"}
]
print(credential_attributes)

[{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]


## Send Credential

This sends a credential to a holder (Bob), and automates the rest of the protocol. 

There are other ways to issue a credential that require multiple api calls.

**Arguments**
* connection_id: The connection_id of the holder you wish to issue to (MUST be in active state)
* schema_id: The id of the schema you wish to issue
* cred_def_id: The definition (public key) to sign the credential object. This must refer to the schema_id and be written to the ledger by the same public did that is currently being used by the agent.
* attributes: A list of attribute objects as defined above. Must match the schema attributes.
* comment (optional): Any string, defaults to ""
* auto_remove (optional): Boolean, defaults to True. I believe this removes the record of this credential once it has been issued. (TODO: double check)
* trace (optional): Boolean, defaults to False. **Not entirely sure about this one, believe its for logging. Also when set to True it throws an error**

In [11]:
print(credential_attributes)
response = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, trace=False)
print(response)

[{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]
{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP', 'auto_remove': True, 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]}, 'connection_id': 'ad11f6e5-351d-459f-aedc-f571ab374fa4', 'trace': False, 'comment': '', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'schema_name': 'open_mined_contributor', 'schema_version': '0.0.1', 'schema_issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}
{'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'auto_offer': False, 'credential_proposal_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/propose-credential', '@id': 'b8924740-5fd4-4465-8d86-f9a3f9f25d59', 'schema_name': 'open_mined_contributor', 'schema_version': '0.0.1', 'c

## Get Credential Exchange Records

The state of the protocol is stored in an object called a Credential Exchange. These can be retrieved from the agent either individually by ID or the full list.

In [11]:
response = await agent_controller.issuer.get_records()
print(response)
record_id = response['results'][0]['credential_exchange_id']
print(record_id)

{'results': [{'auto_offer': False, 'created_at': '2020-07-07 14:58:42.712497Z', 'credential_definition_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'initiator': 'self', 'credential_proposal_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/propose-credential', '@id': 'e8795c83-ab82-42a2-9d00-4c3304cc3c01', 'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]}, 'schema_issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'comment': '', 'schema_version': '0.0.1', 'schema_name': 'open_mined_contributor'}, 'role': 'issuer', 'state': 'offer_sent', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'auto_issue': True, 'connection_id': 'f6bfb182-8c02

In [12]:
credential_record = await agent_controller.issuer.get_record_by_id(record_id)
print(credential_record)

ClientResponseError: 404, message='Not Found', url=URL('http://alice-agent:8021/issue-credential/22b59951-89c0-40d5-9d96-fa31093b2d25')

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<run_in_terminal.<locals>.run() done, defined at /opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py:50> exception=UnsupportedOperation('fileno')>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py", line 55, in run
    return func()
  File "/workspace/aries_basic_controller/utils.py", line 120, in <lambda>
    run_in_terminal(lambda: print_ext(*msg, color=color, **kwargs))
  File "/workspace/aries_basic_controller/utils.py", line 103, in print_ext
    print_formatted(FormattedText(msg), **kwargs)
  File "/workspace/aries_basic_controller/utils.py", line 83, in print_formatted
    prompt_toolkit.print_formatted_text(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/shortcuts/utils.py", line 112, in print_formatted_text
    output = get_app_session().output
  File "/opt/cond

## Remove Credential Exchange

It is possible to remove stored credential exchanges from an agent. Typically when you are not interested in the credential being offered, or the credential offer that you have sent has become stale (been ignored?).

In [15]:
response = await agent_controller.issuer.remove_record(record_id)
print(response)

ClientResponseError: 404, message='Not Found', url=URL('http://alice-agent:8021/issue-credential/records/22b59951-89c0-40d5-9d96-fa31093b2d25/remove')

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<run_in_terminal.<locals>.run() done, defined at /opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py:50> exception=UnsupportedOperation('fileno')>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py", line 55, in run
    return func()
  File "/workspace/aries_basic_controller/utils.py", line 120, in <lambda>
    run_in_terminal(lambda: print_ext(*msg, color=color, **kwargs))
  File "/workspace/aries_basic_controller/utils.py", line 103, in print_ext
    print_formatted(FormattedText(msg), **kwargs)
  File "/workspace/aries_basic_controller/utils.py", line 83, in print_formatted
    prompt_toolkit.print_formatted_text(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/shortcuts/utils.py", line 112, in print_formatted_text
    output = get_app_session().output
  File "/opt/cond

In [16]:
response = await agent_controller.issuer.get_records()
print(response)

{'results': []}


In [13]:
response = await agent_controller.terminate()
print(response)

None
